In [1]:
import os, sys, time

sys.path
sys.path.append('./')

import numpy as np

from astropy.io.fits import getdata
from astropy import wcs
from astropy.io import fits
from astropy import units as u
from astropy import constants as con
from astropy.coordinates import SkyCoord

import matplotlib
matplotlib.use('PDF')
%matplotlib inline
import matplotlib.pyplot as plt
import pandas as pd

# import aplpy

from scipy.optimize import curve_fit

In [5]:
    def plot_spectra(
                     freq_array = np.array([]), 
                     intensity_array = np.array([]),
                     figsize         = (5, 3),
                     xlabel          = 'x-axis [arbitrary unit]', xlabel_size = 12, xscale = 'linear',
                     ylabel          = 'y-axis [arbitrary unit]', ylabel_size = 12, yscale = 'linear',
                     fontsize = 12, 
                     plot_ticks     = True,
                       xtick_size = 12, ytick_size = 12,
                     outPDF_filename = 'none',
                     verbose = False
                    ):
        '''
        This is a function to plot 1D spectra.
        
        Input:
            freq_array [1D numpy array] : store the coordinate values in the frequency axis.
            intensity_array [1D numpy array] : store the coordinate values in the intensity axis.
        
        Keywords:
        
        Output:
        
        Example: 
            x = np.array([1, 2, 3, 4, 5])
            y = np.array([1, 1, 3, 3, 1])
            plot_spectra(x, y)
        '''
        
        if verbose == True:
            print('Plotting 1D spectra')
  
        # Initializing figure
        fig = plt.figure(
                         figsize = (figsize[0], figsize[1])
                        )
        ax = fig.add_axes([0.12, 0.1, 0.75, 0.75])
        
        # Set the x/y axis title and legend
        plt.xlabel(xlabel, size = xlabel_size)
        plt.ylabel(ylabel, size = ylabel_size)

        # set plot scale
        plt.xscale(xscale)
        plt.yscale(yscale)
        
        # set label fontsizes
        plt.rc('font', size = fontsize)            # controls default text sizes
        plt.rc('xtick', labelsize = xtick_size)    # fontsize of the tick labels
        plt.rc('ytick', labelsize = ytick_size)    # fontsize of the tick labels
        if (plot_ticks != True):
            fig.axis_labels.hide_x()
            fig.axis_labels.hide_y()
            fig.tick_labels.hide_x()
            fig.tick_labels.hide_y()

        # plot the data
        plt.plot(freq_array, intensity_array,
                 '-', # symbol shape
                 color=(0.7,0.7,0.2, 0.3), # (R, G, B, transparency), ranged between [0, 1]
                 linewidth = 2.0, 
                 label = "TBD"
                )
                    
        # Setting the figure legend 
        plt.legend(loc=1, fontsize=12)
        
        # Setting the plot range
        # plt.xlim( (-20, num_x) )
        # plt.ylim( (-10, 20) )

        if outPDF_filename != 'none':
            fig.save(outPDF_filename, transparent=True)

In [ ]:
# general functions
def gaussian(x,amplitude,mean,stddev):
    return amplitude * exp(-(x-mean)**2/(2*(ttdev**2)) )
                           
def fit_1dgaus():
    '''
    This is a general use function to fit 1D Gaussian profile.
    '''
    print('Fitting 1D  gaussian')
    
    

        

# defining classes
class spectraAna:
    '''
    Class for analyzing image cube. The main purpose of this analysis is to
    stack the un-redshifted/blueshifted spectra.
    
    '''
    
    ### Constructor and destructor
    def __init__(self, fitscubename = ' '):
        self.fitscubename = fitscubename
    
    def __def__(self):
        pass
    
    
    ##function

    ### Methods in this class
    def readfits(self, verbose = False):
        '''
        The task to read the FITS image cube to memory.
        '''
        if verbose == True:
            print('This is the program to read FITS image cube')
            
        if self.fitscubename != ' ':
            if verbose == True:
                print('Reading FITS image cube :' + self.fitscubename)
        
        # import FITS image to HDU
        self.Ihdu = fits.open(fitscubename)
        
        # choose the certain information
        crval1 = self.Ihdu[0].header['crval1']
        crval2 = self.Ihdu[0].header['crval2']
        crval3 = self.Ihdu[0].header['crval3']
        hduwcs = wcs.WCS(self.Ihdu[0].header)
        print(crval3)
        
        plot_spectra()
        
    def get_velocity(self, verbose = False):
        '''
        The task to fit centroid velocity pixel-by-pixel and store to a 2D numpy array.
        
        Input:
        
        Return:
        
        '''
        if verbose == True:
            print('Fitting centroid velocities')
            
        iterators = []
        for i in iterators:
            fit_1dgaus()
            
        # self.velocity_map = np.array()
        
        
    def write_velocity(self, outfile = 'velocity_map.txt', verbose = False):
        '''
        Output velocity map to an ASCII file.
        
        Input:
            outfile [string] : default: velocity_map.txt
        
        '''
        
        if verbose == True:
            print('Outputing velocity map to an ASCII file')
     
    
    def unrbshift_spectra(self, verbose = False):
        '''
        unredshift/blueshift spectra
        '''
        
        if verbose == True:
            print('Unredshifting/blueshifting spectra')
        
        
            
            
    def stack_spectra(self, verbose = False):
        '''
        The task to stack the unredshifted/unblueshifted spectra and plot the stacked spectra
        '''
        
        if verbose == True:
            print('Stacking the unredshifted/unblueshifted spectra')
            
        #plot_spectra()
        

In [ ]:
## Useful function #####################################################
# plot spectra

# fit spectra (to derive centroid velocity)


## Major flow of data analysis #########################################

# Open FITS image cube
fitscubename = '/home/tcc/DATA/GC_Dspecies/ancillary/Image.GC.CI.image.fits'
myAnalysis = spectraAna(fitscubename = fitscubename)
myAnalysis.readfits(verbose = True)
print(myAnalysis.Ihdu[0].data[0])

# Loop over all spatial pixels and fit the centroid velocity
myAnalysis.get_velocity(verbose = True)

# Output the 2D centroid-velocity map (?)
myAnalysis.write_velocity()

# Based on the 2D centroid-velocity map to un-redshift/blueshift spectra
myAnalysis.unrbshift_spectra()

# stack spectra
myAnalysis.stack_spectra()
